<a href="https://colab.research.google.com/github/AnitaTasnim/gtr_task/blob/main/Task_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#fghhh


In [1]:
!pip install yfinance pandas numpy matplotlib


#vcvcvc

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np

class GoldenCrossTrader_:
    def __init__(self, symbol: str, start_date: str, end_date: str, capital: float = 5000):
        self.symbol = symbol
        self.start_date = start_date
        self.end_date = end_date
        self.capital = capital
        self.data = None
        self.position = False
        self.shares = 0
        self.cash = capital
        self.trades = []

    #      Download Data
    def download_data(self):
        df = yf.download(self.symbol, start=self.start_date, end=self.end_date, progress=False)
        df = df.drop_duplicates().ffill()
        self.data = df
        print(f"[INFO] Downloaded {len(df)} rows for {self.symbol}")

    #      MAs
    def prepare_data(self):
        self.data["MA50"] = self.data["Close"].rolling(window=50).mean()
        self.data["MA200"] = self.data["Close"].rolling(window=200).mean()
        self.data.dropna(inplace=True)
        print("[INFO] Moving averages calculated")

    #  signals
    def generate_signals(self):
        self.data["Signal"] = np.where(self.data["MA50"] > self.data["MA200"], 1, 0)
        self.data["Position"] = self.data["Signal"].diff().fillna(0)
        print("[INFO] Signals generated")

        # Simulate Trades
    def simulate_trades(self):
        prev_price = None
        for date, row in self.data.iterrows():
            #  safer element access
            price = float(row["Close"].item()) if hasattr(row["Close"], "item") else float(row["Close"])
            signal = int(row["Position"].item()) if hasattr(row["Position"], "item") else int(row["Position"])

            # BUY (Golden Cross)
            if signal == 1 and not self.position:
                self.shares = int(self.cash // price)
                if self.shares > 0:
                    cost = self.shares * price
                    self.cash -= cost
                    self.position = True
                    prev_price = price
                    self.trades.append((date, "BUY", price, self.shares, self.cash))

            # SELL (Death Cross)
            elif signal == -1 and self.position:
                proceeds = self.shares * price
                self.cash += proceeds
                profit = (price - prev_price) * self.shares if prev_price else 0
                self.trades.append((date, "SELL", price, self.shares, self.cash, profit))
                self.position = False
                print(f"[SELL] {self.shares} shares at {price:.2f} on {date.date()} | Cash: {self.cash:.2f} | Profit: {profit:.2f}")
                self.shares = 0

        # Force close any open position
        if self.position:
            final_price = float(self.data.iloc[-1]["Close"].item()) if hasattr(self.data.iloc[-1]["Close"], "item") else float(self.data.iloc[-1]["Close"])
            proceeds = self.shares * final_price
            self.cash += proceeds
            profit = (final_price - prev_price) * self.shares if prev_price else 0
            self.trades.append((self.data.index[-1], "FORCE_SELL", final_price, self.shares, self.cash, profit))
            print(f"[FORCE SELL] Closed remaining position at {final_price:.2f} | Profit: {profit:.2f}")
            self.position = False